<a href="https://colab.research.google.com/github/marco-siino/text_preprocessing_impact/blob/main/20N_DS/CNN_20N_TextPreProImpact_NBx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Text preprocessing worth the time: A comparative survey on the impact of common techniques on NLP model performances.
- - -
CNN ON 20N DS EXPERIMENTS NOTEBOOK
- - -
Convolutional Neural Network on 20Newsgroup Dataset.
Code by M. Siino.

From the paper: "Text preprocessing worth the time: A comparative survey on the impact of common techniques on NLP model performances." by M.Siino et al.



## Importing modules.

In [ ]:
import matplotlib.pyplot as plt
import os
import random
import re
import shutil
import string
import tensorflow as tf
import nltk
import numpy as np

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from keras.models import Model
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from textblob import TextBlob
nltk.download('stopwords')
nltk.download('punkt')

os.environ['TF_CUDNN_DETERMINISTIC']='true'
os.environ['TF_DETERMINISTIC_OPS']='true'

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Domenico\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Domenico\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Importing DS and extract in current working directory.

In [ ]:
urlDataset = "https://github.com/marco-siino/text_preprocessing_impact/raw/main/20N_DS/20news-bydate.tar.gz"

dataset = tf.keras.utils.get_file("20news-bydate.tar.gz", urlDataset,
                                    extract=True, archive_format='tar',cache_dir='.',
                                    cache_subdir='')

test_dir = '20news-bydate-test'
train_dir = '20news-bydate-train'

In [ ]:
def walk_dir(targetdir,topdown=True):
    for root, dirs, files in os.walk(targetdir, topdown):
        for subfolder in dirs:
          #print(subfolder)
          #print(root)
          #print(dirs)
          #print(files)
          for subroot, subdirs, files in os.walk(root+'/'+subfolder, topdown):
            #print(files)
            for name in files:
              #print(name)
              os.rename(root+'/'+subfolder+'/'+name, root+'/'+subfolder+'/'+name+'.txt')

walk_dir(test_dir)
walk_dir(train_dir)

FileExistsError: ignored

## Generate full training set.

In [ ]:
# Generate full randomized training set.
batch_size=1

train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    shuffle=False
    )

test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    test_dir,
    batch_size=batch_size,
    shuffle=False
    )

train_ds=train_ds.shuffle(11314,seed=1, reshuffle_each_iteration=False)
test_ds=test_ds.shuffle(7532,seed=1, reshuffle_each_iteration=False)

train_ds_size=len(train_ds)
test_ds_size=len(test_ds)

train_ds = train_ds.take(200)
test_ds = test_ds.take(50)


#train_ds = train_ds.map(lambda x, y: (x, tf.one_hot(y, depth=20)))
#test_ds = test_ds.map(lambda x, y: (x, tf.one_hot(y, depth=20)))

Found 11314 files belonging to 20 classes.
Found 7532 files belonging to 20 classes.


## Functions to pre-process source text. (A detailed discussion on our paper)

In [ ]:
# Do-Nothing preprocessing function.
def DON(input_data):
  tag_open_CDATA_removed = tf.strings.regex_replace(input_data, '<\!\[CDATA\[', ' ')
  tag_closed_CDATA_removed = tf.strings.regex_replace(tag_open_CDATA_removed,'\]{1,}>', ' ')
  tag_author_lang_en_removed = tf.strings.regex_replace(tag_closed_CDATA_removed,'<author lang="en">', ' ')
  tag_closed_author_removed = tf.strings.regex_replace(tag_author_lang_en_removed,'</author>', ' ')
  tag_open_documents_removed = tf.strings.regex_replace(tag_closed_author_removed,'<documents>\n(\t){0,2}', '')
  output_data = tf.strings.regex_replace(tag_open_documents_removed,'</documents>\n(\t){0,2}', ' ')
  return output_data

# Lowercasing preprocessing function.
def LOW(input_data):
  return tf.strings.lower(DON(input_data))

# Removing Stop Words function.
def RSW(input_data):
  output_data = DON(input_data)

  #print("\n\nInput data è il seguente tensore:")
  #print(output_data)

  #print("Lo converto in stringa e diventa:")
  # Il seguente try per l'adattamento del ts. Nell'except caso della simulazione vera e propria.
  try:
    input_string=output_data[0]

  # # # # # # # Questo è il caso della chiamata a funzione per la simulazione vera e propria.
  except:
    #print("\n\n****CASO DELLA SIMULAZIONE VERA E PROPRIA****\n\n")
    #print("\nQuesto è il contenuto di output data in caso di simulazione")
    #print(output_data)
    input_string=output_data

    try:
      input_string = input_string.numpy()

    except:
      #print("This one is not a tensor!")
      return output_data

    else:
      #print("\nEstraendo il contenuto del tensore risulta:")
      input_string=(str(input_string))[2:-1]

    #print(input_string)
    blob = TextBlob(str(input_string)).words

    outputlist = [word for word in blob if word not in stopwords.words('english')]
    #print("tolte le stopword inglesi diventa:")

    output_string = (' '.join(word for word in outputlist))
    #print(output_string)

    output_tensor=tf.constant(output_string)
    #print(output_tensor)

    return output_tensor

   # # # # # # # Questo è il caso dell'adattamento del TS.
  else:

    try:

      # input_string = input_string.numpy() [0]
      input_string = input_string.numpy()

    except:
      #print("This one is not a tensor!")
      return output_data

    else:
      input_string=(str(input_string))[2:-1]

    #print(input_string)
    blob = TextBlob(str(input_string)).words

    outputlist = [word for word in blob if word not in stopwords.words('english')]
    #print("Tolte le stopword inglesi diventa:")

    output_string = (' '.join(word for word in outputlist))
    #print(output_string)

    output_tensor=tf.constant([[output_string]])
    #print(output_tensor)

    return output_tensor

  return output_data

# Porter Stemmer preprocessing function.
def STM(input_data):
  output_data = DON(input_data)
  stemmer = PorterStemmer()

  #print("\n\nInput data è il seguente tensore:")
  #print(output_data)

  #print("Lo converto in stringa e diventa:")
  # Il seguente try per l'adattamento del ts. Nell'except caso della simulazione vera e propria.
  try:
    input_string=output_data[0]

  # # # # # # # Questo è il caso della chiamata a funzione per la simulazione vera e propria.
  except:
    #print("\n\n****CASO DELLA SIMULAZIONE VERA E PROPRIA****\n\n")
    #print("\nQuesto è il contenuto di output data in caso di simulazione")
    #print(output_data)
    input_string=output_data

    try:
      input_string = input_string.numpy()

    except:
      #print("This one is not a tensor!")
      return output_data

    else:
      #print("\nEstraendo il contenuto del tensore risulta:")
      #print(input_string)
      input_string=(str(input_string))[2:-1]

    #print(input_string)
    blob = TextBlob(str(input_string)).words

    outputlist = [stemmer.stem(word) for word in blob]

    output_string = (' '.join(word for word in outputlist))
    #print(output_string)

    output_tensor=tf.constant(output_string)
    #print(output_tensor)

    return output_tensor

   # # # # # # # Questo è il caso dell'adattamento del TS.
  else:

    try:
      #input_string = input_string.numpy()[0]
      input_string = input_string.numpy()
      #print(input_string)

    except:
      #print("This one is not a tensor!")
      return output_data

    else:
      input_string=(str(input_string))[2:-1]

    #print(input_string)
    blob = TextBlob(str(input_string)).words

    outputlist = [stemmer.stem(word) for word in blob]

    output_string = (' '.join(word for word in outputlist))

    output_tensor=tf.constant([[output_string]])
    #print(output_tensor)

    return output_tensor

  return output_data

## Define the combined preprocessing functions. (The base functions are: DON, LOW, RSW and STM).

In [ ]:
## SECTION WITH PAIRS OF PREPRO FUNCTIONS. APPLICATION ORDER MATTERS (...IN FOLLOWING SECTIONS TOO).
#...5
def LOW_RSW(input_data):
  return RSW(LOW(input_data))

# 6
def LOW_STM(input_data):
  return STM(LOW(input_data))

# 7
def RSW_LOW(input_data):
  return LOW(RSW(input_data))

# 8
def RSW_STM(input_data):
  return STM(RSW(input_data))

# 9
def STM_LOW(input_data):
  return LOW(STM(input_data))

# 10
def STM_RSW(input_data):
  return RSW(STM(input_data))

# 11
def LOW_STM_RSW(input_data):
  return RSW(STM(LOW(input_data)))

# 12
def LOW_RSW_STM(input_data):
  return STM(RSW(LOW(input_data)))

# 13
def STM_LOW_RSW(input_data):
  return RSW(LOW(STM(input_data)))

# 14
def STM_RSW_LOW(input_data):
  return LOW(RSW(STM(input_data)))

# 15
def RSW_LOW_STM(input_data):
  return STM(LOW(RSW(input_data)))

# 16
def RSW_STM_LOW(input_data):
  return LOW(STM(RSW(input_data)))

## Get the length of the longest sample in training set. Then adapt text.



In [ ]:
max_features = 0
def preprocess_and_adapt_ts(preprocessing_function,training_set):
  # Set a large sequence length to find the longest sample in the training set.
  sequence_length = 15000
  vectorize_layer = TextVectorization(
      standardize=preprocessing_function,
      output_mode='int',
      output_sequence_length=sequence_length,
      encoding='ISO-8859-1')

  train_text = training_set.map(lambda x, y: x)
  vectorize_layer.adapt(train_text)
  #vectorize_layer.get_vocabulary()

  model = tf.keras.models.Sequential()
  model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
  model.add(vectorize_layer)

  longest_sample_length=1

  for element in training_set:
    authorDocument=element[0]
    label=element[1]

    #print("Sample considered is: ", authorDocument[0].numpy())
    #print("Preprocessed: ", str(custom_standardization(authorDocument[0].numpy())))
    #print("And has label: ", label[0].numpy())

    out=model(authorDocument)
    # Convert token list to numpy array.
    token_list = out.numpy()[0]
    token_list = np.trim_zeros(token_list,'b')
    if longest_sample_length < len(token_list):
      longest_sample_length = len(token_list)

  print("Length of the longest sample is:", longest_sample_length)

  # After tokenization longest_sample_length covers all the document lenghts in our dataset.
  sequence_length = longest_sample_length

  vectorize_layer = TextVectorization(
      standardize=preprocessing_function,
      output_mode='int',
      output_sequence_length=sequence_length,
      encoding='ISO-8859-1')

  # Finally adapt the vectorize layer.
  train_text = training_set.map(lambda x, y: x)
  vectorize_layer.adapt(train_text)
  global max_features
  max_features=len(vectorize_layer.get_vocabulary()) + 1
  return vectorize_layer

## Define a dictionary with -> function_names:prepro_function_caller. And a dictionary to store model results.




In [ ]:
model_results = {}
prepro_functions_dict_base = {
    'DON':DON,
    'LOW':LOW,
    'RSW':RSW,
    'STM':STM
    }

# 3 prepro functions = 15 combs...+1 for do_nothing

prepro_functions_dict_comb = {
    # 1. Do nothing
    'DON': DON,
    # 2. Lowercasing
    'LOW':LOW,
    # 3. Removing Stopwords
    'RSW':RSW,
    # 4. Porter Stemming
    'STM':STM,
    # 5. LOW->RSW
    'LOW_RSW':LOW_RSW,
    # 6. LOW->STM
    'LOW_STM':LOW_STM,
    # 7. RSW->LOW
    'RSW_LOW':RSW_LOW,
    # 8. RSW->STM
    'RSW_STM':RSW_STM,
    # 9. STM->LOW
    'STM_LOW':STM_LOW,
    # 10. STM->RSW
    'STM_RSW':STM_RSW,
    # 11. LOW->STM->RSW
    'LOW_STM_RSW':LOW_STM_RSW,
    # 12. LOW->RSW->STM
    'LOW_RSW_STM':LOW_RSW_STM,
    # 13. STM->LOW->RSW
    'STM_LOW_RSW':STM_LOW_RSW,
    # 14. STM->RSW->LOW
    'STM_RSW_LOW':STM_RSW_LOW,
    # 15. RSW->LOW->STM
    'RSW_LOW_STM':RSW_LOW_STM,
    # 16. RSW->STM->LOW
    'RSW_STM_LOW':RSW_STM_LOW
}

for key in prepro_functions_dict_comb:
  print(key)
  model_results[key]=[]

DON
LOW
RSW
STM
LOW_RSW
LOW_STM
RSW_LOW
RSW_STM
STM_LOW
STM_RSW
LOW_STM_RSW
LOW_RSW_STM
STM_LOW_RSW
STM_RSW_LOW
RSW_LOW_STM
RSW_STM_LOW


## Some training hyperparameters...

In [ ]:
# Word embedding dimensions.
embedding_dim = 100

num_runs = 5
# No need to go over the 20th epoch...Overfitting begins.
num_epochs_per_run = 50

## Models definition and evaluation.




In [ ]:
tf.random.set_seed(1)

# Reset model_results list.
for key in prepro_functions_dict_comb:
  model_results[key]=[]

for key in prepro_functions_dict_comb:
  runs_accuracy = []

  print("\n\n* * * * EVALUATION USING", key, "AS PREPROCESSING FUNCTION * * * *")

  # Preprocess training set to build a dictionary.
  vectorize_layer = preprocess_and_adapt_ts(prepro_functions_dict_comb[key],train_ds)

  #max_features=len(vectorize_layer.get_vocabulary()) + 1
  print("Vocabulary size is:", max_features)

  for run in range(1,(num_runs+1)):
    opt = tf.keras.optimizers.Adam()
    epochs_accuracy=[]
    model = tf.keras.Sequential([
                                    tf.keras.Input(shape=(1,), dtype=tf.string),
                                    vectorize_layer,
                                    layers.Embedding(max_features + 1, embedding_dim),
                                    layers.Dropout(0.8),

                                    layers.Conv1D(256,16,activation='relu'),
                                    layers.MaxPooling1D(),
                                    layers.Dropout(0.6),

                                    layers.Dense(512,activation='relu'),

                                    layers.GlobalAveragePooling1D(),
                                    layers.Dropout(0.2),
                                    layers.Dense(20,activation="softmax")
    ])
    model.compile( loss='sparse_categorical_crossentropy', optimizer=opt, metrics=["acc"])

    for epoch in range (0,num_epochs_per_run):
        history = model.fit(
          train_ds,
          validation_data = test_ds,
          batch_size = 1,
          epochs=1,
          shuffle=False,
          # Comment the following line to do not save and download the model.
          #callbacks=[callbacks]
          )
        accuracy = history.history['acc']
        print("Run: ",run,"/ Accuracy at epoch ",epoch," is: ", accuracy[0],"\n")
        epochs_accuracy.append(accuracy[0])

    print("Accuracies over epochs:",epochs_accuracy,"\n\n")
    runs_accuracy.append(max(epochs_accuracy))

  runs_accuracy.sort()
  print("\n\n Over all runs maximum accuracies are:", runs_accuracy)
  print("The median is:",runs_accuracy[2],"\n\n\n")

  if (runs_accuracy[2]-runs_accuracy[0])>(runs_accuracy[4]-runs_accuracy[2]):
    max_range_from_median = runs_accuracy[2]-runs_accuracy[0]
  else:
    max_range_from_median = runs_accuracy[4]-runs_accuracy[2]
  final_result = str(runs_accuracy[2])+" +/- "+ str(max_range_from_median)
  model_results[key].append(final_result)
  print("ANN Accuracy Score on Test set -> ",model_results[key])



* * * * EVALUATION USING DON AS PREPROCESSING FUNCTION * * * *
Length of the longest sample is: 7969
Vocabulary size is: 17227
200/200 [==============================] - 35s 157ms/step - loss: 3.0187 - acc: 0.0550 - val_loss: 2.9974 - val_acc: 0.0400
Run:  1 / Accuracy at epoch  0  is:  0.054999999701976776 

200/200 [==============================] - 34s 151ms/step - loss: 2.9887 - acc: 0.0800 - val_loss: 3.0015 - val_acc: 0.0400
Run:  1 / Accuracy at epoch  1  is:  0.07999999821186066 

200/200 [==============================] - 33s 145ms/step - loss: 2.9689 - acc: 0.0850 - val_loss: 3.0030 - val_acc: 0.0400
Run:  1 / Accuracy at epoch  2  is:  0.08500000089406967 

200/200 [==============================] - 32s 148ms/step - loss: 2.8609 - acc: 0.0750 - val_loss: 3.0058 - val_acc: 0.0600
Run:  1 / Accuracy at epoch  3  is:  0.07500000298023224 

200/200 [==============================] - 34s 151ms/step - loss: 2.7848 - acc: 0.0800 - val_loss: 3.0120 - val_acc: 0.0600
Run:  1 / Accu

In [ ]:
i=1

for sample in train_ds:
  print("\n\n")
  print(sample[1])
  print(model.predict(sample[0]))
  i+=1
  if i==100:
    break




tf.Tensor([5], shape=(1,), dtype=int32)
1/1 [==============================] - 0s 276ms/step
[[3.7495283e-33 5.2402940e-25 1.5168798e-06 3.9887220e-02 4.9482566e-17
  9.4338769e-01 4.0977805e-22 3.2565894e-11 9.2028435e-03 1.8751681e-04
  1.6074642e-08 4.5206747e-04 6.7888573e-03 8.7117543e-05 2.2895868e-07
  6.1916279e-14 3.2365749e-06 1.2478346e-06 4.7020333e-07 4.3965022e-19]]



tf.Tensor([11], shape=(1,), dtype=int32)
1/1 [==============================] - 0s 52ms/step
[[5.7339889e-28 1.3837441e-11 5.6237262e-02 4.2135321e-07 4.7394100e-10
  1.9256908e-02 4.5397815e-14 5.5282807e-04 6.0707539e-06 4.9276931e-16
  0.0000000e+00 9.2393547e-01 1.8457558e-14 9.3799263e-06 3.2958801e-14
  3.9149241e-22 4.1444167e-27 1.8780974e-25 1.6320837e-06 1.9718507e-18]]



tf.Tensor([3], shape=(1,), dtype=int32)
1/1 [==============================] - 0s 49ms/step
[[0.00000000e+00 0.00000000e+00 1.00223358e-22 9.77505147e-01
  0.00000000e+00 2.57425476e-03 0.00000000e+00 1.05754694e-32
  3.01528

## Now show compact results in a table.

In [ ]:
print(" PREPRO FUNCTION    |  Test Accuracy   |",end = '')

print("\n")
for prepro_func in prepro_functions_dict_comb:
  #print(prepro_func,"\t\t\t",format(round(model_results[prepro_func][0],4),'.4f'),"\t\t",end='')
  result = model_results[prepro_func][0]
  # result = format(round(model_results[prepro_func][0],4),'.4f')
  print(f'{prepro_func:27}{ result :12}')
  print("\n")
